In [2]:
import pandas as pd

In [3]:
import math

def harmonize_agrupacion_id(agrupacion_id):
    if pd.isna(agrupacion_id):
        return "000000"
    else:
        try:
            # Attempt to convert string to float and then to integer
            return str(int(float(agrupacion_id))).zfill(6)
        except ValueError:
            # If conversion fails, return original string value
            return agrupacion_id

In [4]:
eleccion = pd.read_csv('./../datos/BD/eleccion_table.csv')
eleccion

,eleccion_id,año,eleccion_tipo,recuento_tipo,padron_tipo
0,1,2011,PASO,PROVISORIO,NORMAL
1,10,2017,PASO,PROVISORIO,NORMAL
2,9,2017,PASO,PROVISORIO,COMANDO
3,8,2017,GENERAL,PROVISORIO,NORMAL
4,7,2017,GENERAL,PROVISORIO,COMANDO
5,5,2015,GENERAL,PROVISORIO,NORMAL
6,4,2015,BALLOTAGE,PROVISORIO,NORMAL
7,0,2011,GENERAL,PROVISORIO,NORMAL
8,6,2015,PASO,PROVISORIO,NORMAL
9,3,2013,PASO,PROVISORIO,NORMAL


In [5]:
import pandas as pd

In [6]:
cargo = pd.read_csv('./../datos/BD/cargo_table.csv')

agrup_lista = pd.read_csv('./../datos/BD/agrupacion_lista_table.csv')
agrup_lista['agrupacion_id'] = agrup_lista['agrupacion_id'].apply(harmonize_agrupacion_id)
agrup_lista['lista_numero'] = agrup_lista['lista_numero'].apply(harmonize_agrupacion_id)
agrup_lista

agrup = agrup_lista[['eleccion_id',	'agrupacion_id']].drop_duplicates()
lista = agrup_lista[['eleccion_id',	'agrupacion_id', 'lista_numero']].drop_duplicates()

agrup

,eleccion_id,agrupacion_id
0,1,000509
4,1,000003
8,1,000037
11,1,000133
12,1,000057
...,...,...
5749,12,02-737
5750,12,02-1912
5751,12,02-884
5752,12,02-888


In [7]:
# agrup_lista.groupby(['agrupacion_id'])['agrupacion_nombre'].unique().apply(lambda x: x[0:2]).to_frame().reset_index()

# Si queremos un solo valor por agrupacion id
# agrup_names = agrup_lista.groupby(['eleccion_id', 'distrito_id', 'agrupacion_id']).agrupacion_nombre.unique().astype(str).reset_index()


In [8]:
fuerzas = pd.read_csv('./../datos/info/agrupacion_nombre_weight_labels.csv')

agrup_fzas = agrup_lista.merge(fuerzas, how = 'left')
agrup_fzas.loc[agrup_fzas.agrupacion_id == '000000', 'fuerza'] = ''
agrup_fzas['fuerza'] = agrup_fzas['fuerza'].fillna('OTROS')
fuerzas = agrup_fzas[['agrupacion_nombre', 'fuerza']].drop_duplicates()

In [9]:
import glob
import os

## list of csv files
csv_files = glob.glob('./../datos/BD/votos_eleccion_*_table.csv')

## list of dataframes
dfs = [pd.read_csv(f) for f in csv_files]
data = pd.concat(dfs, ignore_index=True)

data['agrupacion_id'] = data['agrupacion_id'].apply(harmonize_agrupacion_id)
# data['lista_numero'] = data['lista_numero'].apply(harmonize_agrupacion_id)
# agrup_lista.lista_numero.value_counts()

/tmp/ipykernel_18735/2812810793.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_18735/2812810793.py:8: DtypeWarning: Columns (3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_18735/2812810793.py:8: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_18735/2812810793.py:8: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_18735/2812810793.py:8: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs = [pd.read_csv(f) for f in csv_files]
/tmp/ipykernel_18735/2812810793.py:8: DtypeWarning: Columns (3,7,9) have mixed types. 

## Votos por agrupacion, provincia, eleccion, cargo

In [10]:
data.votos_cantidad.sum()

850253087

In [11]:
# Mezclar los datos con un subconjunto de columnas de agrup_lista y luego eliminar los duplicados
# merged_data = data.merge(agrup_lista[['eleccion_id', 'agrupacion_id', 'agrupacion_nombre']].drop_duplicates())
simil_nombre = agrup_lista.groupby(['eleccion_id', 'distrito_id', 'agrupacion_id']).agrupacion_nombre.first().reset_index()
merged_data = data.merge(simil_nombre).merge(fuerzas)

# Mezclar el conjunto de datos resultante con el conjunto de datos de cargo
merged_data = merged_data.merge(cargo)

# Agrupar el conjunto de datos resultante por varias columnas y agregar la columna votos_cantidad sumando sus valores
grouped_data = merged_data.groupby(['eleccion_id', 'cargo_id', 'fuerza']).agg({'votos_cantidad': 'sum'})

# Restablecer el índice del conjunto de datos resultante y crear un nuevo índice
table = grouped_data.unstack('fuerza')/1e3
table.round(1) # miles


votos_cantidad                                           \
fuerza                              IZQUIERDA MASSA   OTROS    PERON     PRO   
eleccion_id cargo_id                                                           
0           1                 908.7     497.1   NaN  1660.8  11593.0     NaN   
            2                 977.4     279.4   NaN   998.4   5310.6     NaN   
            3                1968.4     494.7   NaN  2688.3   9871.4   368.0   
            4                1352.4     244.5   NaN  1478.7   5033.6     NaN   
            5                 684.6     132.2   NaN   780.9   2669.0     NaN   
...                             ...       ...   ...     ...      ...     ...   
16          3                1472.6    1129.4  49.8  3749.9   6480.1  8599.0   
            5                 467.7     289.4   NaN   838.2   1633.4  2155.2   
            6                 574.6     443.6   NaN  1249.9   2523.7  3045.7   
            7                   0.2       NaN   NaN     2.8      2.7     0.2   
            10                814.9     539.0   NaN  1560.4   3240.2  3599.0   

                                                    
fuerza               PROVINCIAL SOCIALISTA     UCR  
eleccion_id cargo_id                                
0           1            1714.4     3624.5  2395.1  
            2             910.4     1078.3  1167.0  
            3            1676.7     2616.2  2443.4  
            4             125.3     1013.2  1273.9  
            5             267.1      548.6   458.7  
...                         ...        ...     ...  
16          3            1227.8       50.1     NaN  
            5              32.5        NaN     NaN  
            6              32.5        NaN     NaN  
            7               NaN        NaN     NaN  
            10             34.0        NaN     NaN  

[98 rows x 9 columns]

In [12]:
table.votos_cantidad.sum()

fuerza
               63999.189
IZQUIERDA      28451.893
MASSA          66209.025
OTROS          75686.495
PERON         319551.758
PRO           223334.396
PROVINCIAL     37392.734
SOCIALISTA     17216.743
UCR            17741.231
dtype: float64

In [27]:
eleccion_tags = pd.read_csv('./../datos/BD/eleccion_tags.csv')
distritos = pd.read_csv('./../datos/BD/distrito_table.csv')


In [22]:
## Tabla para verificar que los ppales agrupaciones de cada provincia tienen bien colocado el nombre de fuerza

In [44]:
# # # data = merged_data.loc[merged_data.cargo_id.isin([1, 3])].copy()

# # # votos_prov_agrup = data.groupby(['eleccion_id', 'cargo_nombre', 'distrito_id', 'agrupacion_nombre']).votos_cantidad.sum()

# votos_prov_agrup = votos_prov_agrup.reset_index().merge(eleccion_tags).merge(distritos)
# votos_prov_agrup = votos_prov_agrup[['año', 'eleccion_tipo', 'padron_tipo', 'distrito_nombre', 'cargo_nombre', 'agrupacion_nombre', 'votos_cantidad']]
# main_agrupaciones = votos_prov_agrup.sort_values('votos_cantidad', ascending=False).groupby(['año', 'eleccion_tipo', 'padron_tipo', 'distrito_nombre', 'cargo_nombre']).head(3)
# tabla_checks = main_agrupaciones.merge(fuerzas).sort_values(['año', 'eleccion_tipo', 'padron_tipo', 'distrito_nombre', 'cargo_nombre'])
# tabla_checks.to_csv('./../datos/info/agrupacion_fuerza_checks.csv', index=False)
# tabla_checks.head(12)

,año,eleccion_tipo,padron_tipo,distrito_nombre,cargo_nombre,agrupacion_nombre,votos_cantidad,fuerza
179,2011,GENERAL,NORMAL,Buenos Aires,Diputado Nacional,Alianza Frente para la Victoria,4460986,PERON
638,2011,GENERAL,NORMAL,Buenos Aires,Diputado Nacional,Alianza Frente Amplio Progresista,1018572,SOCIALISTA
675,2011,GENERAL,NORMAL,Buenos Aires,Diputado Nacional,Alianza Unión para el Desarrollo Social,905064,UCR
178,2011,GENERAL,NORMAL,Buenos Aires,Presidente,Alianza Frente para la Victoria,4704016,PERON
637,2011,GENERAL,NORMAL,Buenos Aires,Presidente,Alianza Frente Amplio Progresista,1251277,SOCIALISTA
676,2011,GENERAL,NORMAL,Buenos Aires,Presidente,Alianza Unión para el Desarrollo Social,810882,UCR
211,2011,GENERAL,NORMAL,Catamarca,Diputado Nacional,Alianza Frente para la Victoria,120806,PERON
661,2011,GENERAL,NORMAL,Catamarca,Diputado Nacional,Alianza Frente Amplio Progresista,7237,SOCIALISTA
1157,2011,GENERAL,NORMAL,Catamarca,Diputado Nacional,Alianza Frente Cívico y Social,39715,SOCIALISTA
210,2011,GENERAL,NORMAL,Catamarca,Presidente,Alianza Frente para la Victoria,130983,PERON


In [13]:
data_prov = merged_data.groupby(['eleccion_id', 'cargo_id', 'fuerza', 'distrito_id']).agg({'votos_cantidad': 'sum'})
data_prov.to_csv('./../datos/out/votos_prov.csv')
data_prov.votos_cantidad.sum()
# 849583464

849583464

In [14]:
data_secc = merged_data.groupby(['eleccion_id', 'cargo_id', 'fuerza', 'distrito_id', 'seccion_id']).agg({'votos_cantidad': 'sum'})
data_secc.to_csv('./../datos/out/votos_secc.csv')
data_secc.votos_cantidad.sum()

849583464

In [15]:
## Seccion_id no es necesario, alcanza con provincia-circuito. Pero lo agregamos porque a traves de los circuitos se relaciona el seccion_id con el id de deptos en IGN.
data_circ = merged_data.groupby(['eleccion_id', 'cargo_id', 'fuerza', 'distrito_id', 'seccion_id', 'circuito_id']).agg({'votos_cantidad': 'sum'})
data_circ.to_csv('./../datos/out/votos_circ.csv')
data_circ.votos_cantidad.sum()

849583464

In [16]:
table.votos_cantidad.sum()

fuerza
               63999.189
IZQUIERDA      28451.893
MASSA          66209.025
OTROS          75686.495
PERON         319551.758
PRO           223334.396
PROVINCIAL     37392.734
SOCIALISTA     17216.743
UCR            17741.231
dtype: float64